In [35]:
from pronto import Ontology
import json
import pandas as pd

In [2]:
do = Ontology('https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/master/src/ontology/HumanDO.obo')

In [26]:
def read_term (i):
    ii = str(i).replace('Term(','').replace(')','').replace("'",'').split(',')
    doid = ii[0].replace('DOID:','')
    return doid
    

In [27]:
def read_xref (j):
    jj = set(j)
    for n in jj:
        xref = str(n).replace('Xref','').replace('(','').replace(')','').replace("'",'').split(':')
        if 'UMLS_CUI' in xref[0]:
            umls = xref[1]
            return umls

In [47]:
def do_umls(do):
    mapp = {}
    for i in do.terms():
        doid = read_term(i)
        umls = read_xref(i.xrefs)
        if umls is not None:
            mapp[umls] = doid
    return mapp


In [48]:
do2umls = do_umls(do)
with open('do2umls.json','w') as f:
    json.dump(do2umls,f)

In [66]:
repo = pd.read_csv('/home/lxu/Downloads/full.csv')

In [67]:
repo.head()

,drug_name,drug_id,ind_name,ind_id,NCT,status,phase,DetailedStatus
0,Lepirudin,DB00001,Heparin-induced thrombocytopenia with thrombosis,C0272275,NaN,Approved,NaN,NaN
1,Cetuximab,DB00002,Squamous cell carcinoma of mouth,C0585362,NaN,Approved,NaN,NaN
2,Cetuximab,DB00002,Squamous cell carcinoma of nose,C3163899,NaN,Approved,NaN,NaN
3,Cetuximab,DB00002,Squamous cell carcinoma of pharynx,C1319317,NaN,Approved,NaN,NaN
4,Cetuximab,DB00002,Laryngeal Squamous Cell Carcinoma,C0280324,NaN,Approved,NaN,NaN


In [70]:
doid = []
for _,row in repo[['ind_id']].iterrows():
    if row['ind_id'] in do2umls.keys():
        
        do_id = do2umls[row['ind_id']]
        
        doid.append(do_id)
    else:
        doid.append(None)
    


In [73]:
repo.insert(4,'do_id',doid)


,drug_name,drug_id,ind_name,ind_id,do_id,NCT,status,phase,DetailedStatus
0,Lepirudin,DB00001,Heparin-induced thrombocytopenia with thrombosis,C0272275,None,NaN,Approved,NaN,NaN
1,Cetuximab,DB00002,Squamous cell carcinoma of mouth,C0585362,None,NaN,Approved,NaN,NaN
2,Cetuximab,DB00002,Squamous cell carcinoma of nose,C3163899,None,NaN,Approved,NaN,NaN
3,Cetuximab,DB00002,Squamous cell carcinoma of pharynx,C1319317,None,NaN,Approved,NaN,NaN
4,Cetuximab,DB00002,Laryngeal Squamous Cell Carcinoma,C0280324,2876,NaN,Approved,NaN,NaN
5,Cetuximab,DB00002,Malignant tumor of colon,C0007102,219,NaN,Approved,NaN,NaN
6,Dornase alfa,DB00003,Cystic Fibrosis,C0010674,1485,NaN,Approved,NaN,NaN
7,Denileukin diftitox,DB00004,"Lymphoma, T-Cell, Cutaneous",C0079773,None,NaN,Approved,NaN,NaN
8,Etanercept,DB00005,Rheumatoid Arthritis,C0003873,7148,NaN,Approved,NaN,NaN
9,Etanercept,DB00005,Juvenile arthritis,C3495559,None,NaN,Approved,NaN,NaN


In [74]:
nodes = pd.read_csv('/home/lxu/Downloads/integrate-master/data/nodes.tsv','\t')

In [78]:
compound = nodes.loc[nodes['kind']=='Compound']['id'].tolist()
disease = nodes.loc[nodes['kind']=='Disease']['id'].tolist()

In [94]:
drug_in_hetio = []
disease_in_hetio = []
for _, row in repo[['drug_id','do_id']].iterrows():
    if 'Compound::'+row['drug_id'] in compound:
        drug_in_hetio.append(1)
    else:
        drug_in_hetio.append(0)
    if row['do_id'] and 'Disease::DOID:'+row['do_id'] in disease:
        disease_in_hetio.append(1)
    else:
        disease_in_hetio.append(0)
    

In [95]:
disease_in_hetio.count(1)

746

In [96]:
repo.insert(2,'drug_in_hetio',drug_in_hetio)
repo.insert(5,'disease_in_hetio',disease_in_hetio)

In [97]:
repo.head()

,drug_name,drug_id,drug_in_hetio,ind_name,ind_id,disease_in_hetio,do_id,NCT,status,phase,DetailedStatus
0,Lepirudin,DB00001,0,Heparin-induced thrombocytopenia with thrombosis,C0272275,0,None,NaN,Approved,NaN,NaN
1,Cetuximab,DB00002,0,Squamous cell carcinoma of mouth,C0585362,0,None,NaN,Approved,NaN,NaN
2,Cetuximab,DB00002,0,Squamous cell carcinoma of nose,C3163899,0,None,NaN,Approved,NaN,NaN
3,Cetuximab,DB00002,0,Squamous cell carcinoma of pharynx,C1319317,0,None,NaN,Approved,NaN,NaN
4,Cetuximab,DB00002,0,Laryngeal Squamous Cell Carcinoma,C0280324,0,2876,NaN,Approved,NaN,NaN


In [106]:
repo_hetio = []
for _,row in repo[['drug_id','drug_in_hetio','disease_in_hetio','do_id','status']].iterrows():
    if row['drug_in_hetio'] ==1 and row['disease_in_hetio'] == 1:
        if row['status'] == 'Approved':
            drug = 'Compound::' + row['drug_id']
            disease = 'Disease::DOID:' + row['do_id']
            repo_hetio.append((drug,disease,1))
        else:
            repo_hetio.append((drug,disease,0))

In [108]:
df = pd.DataFrame(repo_hetio, columns = ['drug','disease','status'])
df

In [109]:
df_pos = df.loc[df['status']==1]
df_neg = df.loc[df['status']==0]

In [110]:
len(df_pos)

441

In [111]:
len(df_neg)

187